In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from dataclasses import dataclass

# =============================
# Helpers
# =============================
def annual_to_monthly(r: float) -> float:
    """Annual effective rate -> monthly effective rate."""
    return (1.0 + r) ** (1.0 / 12.0) - 1.0

def pct(x: float) -> str:
    if x is None or (isinstance(x, float) and not np.isfinite(x)):
        return "nan"
    return f"{x*100:.2f}%"

def last_close(ticker: str, period: str = "5d") -> float:
    h = yf.Ticker(ticker).history(period=period)
    if h is None or h.empty:
        return np.nan
    return float(h["Close"].iloc[-1])

# =============================
# Core data container
# =============================
@dataclass
class MRPScenario:
    label: str
    earnings_yield: float
    growth: float
    market_return_annual: float
    risk_free_annual: float
    mrp_annual: float
    market_return_monthly: float
    risk_free_monthly: float
    mrp_monthly: float
    pe_used: str
    pe_value: float

# =============================
# Risk-free from Yahoo
# =============================
def get_risk_free_rate(rf_ticker: str = "^IRX") -> float:
    """
    ^IRX quoted in percent (e.g., 4.55 means 4.55% annualized)
    Returns decimal (0.0455).
    """
    rf_pct = last_close(rf_ticker)
    if not np.isfinite(rf_pct):
        return np.nan
    return rf_pct / 100.0

# =============================
# Earnings yield from PE
# =============================
def get_earnings_yield_from_yahoo(market_proxy: str = "SPY") -> tuple[float, str, float]:
    """
    Returns:
      (earnings_yield, pe_used_name, pe_value)

    Priority:
      1) forwardPE -> EY = 1/forwardPE
      2) trailingPE -> EY = 1/trailingPE
    """
    t = yf.Ticker(market_proxy)
    info = t.info or {}

    # forward PE preferred
    fpe = info.get("forwardPE", None)
    if isinstance(fpe, (int, float)) and np.isfinite(fpe) and fpe > 0:
        return (1.0 / float(fpe), "forwardPE", float(fpe))

    # trailing PE fallback
    tpe = info.get("trailingPE", None)
    if isinstance(tpe, (int, float)) and np.isfinite(tpe) and tpe > 0:
        return (1.0 / float(tpe), "trailingPE", float(tpe))

    return (np.nan, "none", np.nan)

# =============================
# Main: Earnings Yield Approach
# =============================
def calc_mrp_earnings_yield_approach(
    market_proxy: str = "SPY",
    rf_proxy: str = "^IRX",
    growth_base: float = 0.045,
    growth_low: float = 0.030,
    growth_high: float = 0.060,
) -> pd.DataFrame:
    """
    Earnings Yield approach:
      E(Rm) = EY + g
      MRP   = E(Rm) - Rf

    Returns DataFrame with Low/Base/High scenarios.
    """
    ey, pe_used, pe_value = get_earnings_yield_from_yahoo(market_proxy)
    rf = get_risk_free_rate(rf_proxy)

    if not np.isfinite(ey):
        raise ValueError(
            f"Earnings Yield unavailable from Yahoo for {market_proxy}. "
            "Try a different proxy or use another data source for index-level forward PE."
        )

    if not np.isfinite(rf):
        raise ValueError(f"Risk-free rate unavailable from Yahoo for {rf_proxy}.")

    scenarios = [
        ("LOW",  growth_low),
        ("BASE", growth_base),
        ("HIGH", growth_high),
    ]

    rows = []
    for label, g in scenarios:
        er_m_annual = ey + g
        mrp_annual = er_m_annual - rf

        er_m_monthly = annual_to_monthly(er_m_annual)
        rf_monthly = annual_to_monthly(rf)
        mrp_monthly = er_m_monthly - rf_monthly

        rows.append(MRPScenario(
            label=label,
            earnings_yield=ey,
            growth=g,
            market_return_annual=er_m_annual,
            risk_free_annual=rf,
            mrp_annual=mrp_annual,
            market_return_monthly=er_m_monthly,
            risk_free_monthly=rf_monthly,
            mrp_monthly=mrp_monthly,
            pe_used=pe_used,
            pe_value=pe_value
        ))

    df = pd.DataFrame([r.__dict__ for r in rows])

    # Add a nice formatted view columns (optional)
    df["Earnings Yield (E/P)"] = df["earnings_yield"].apply(pct)
    df["Growth (g)"] = df["growth"].apply(pct)
    df["E(Rm) Annual"] = df["market_return_annual"].apply(pct)
    df["Rf Annual"] = df["risk_free_annual"].apply(pct)
    df["MRP Annual"] = df["mrp_annual"].apply(pct)
    df["E(Rm) Monthly"] = df["market_return_monthly"].apply(pct)
    df["Rf Monthly"] = df["risk_free_monthly"].apply(pct)
    df["MRP Monthly"] = df["mrp_monthly"].apply(pct)

    # Order columns (clean)
    df = df[[
        "label",
        "pe_used", "pe_value",
        "Earnings Yield (E/P)",
        "Growth (g)",
        "E(Rm) Annual",
        "Rf Annual",
        "MRP Annual",
        "E(Rm) Monthly",
        "Rf Monthly",
        "MRP Monthly",
        # raw numeric (keep for modeling)
        "earnings_yield", "growth", "market_return_annual", "risk_free_annual", "mrp_annual",
        "market_return_monthly", "risk_free_monthly", "mrp_monthly"
    ]]

    return df

# =============================
# Run
# =============================
if __name__ == "__main__":
    df = calc_mrp_earnings_yield_approach(
        market_proxy="SPY",   # proxy for S&P 500 exposure
        rf_proxy="^IRX",      # 13-week T-bill
        growth_base=0.045,
        growth_low=0.030,
        growth_high=0.060
    )

    print("=== Earnings Yield Approach (S&P 500 proxy) ===")
    print(df[[
        "label", "pe_used", "pe_value",
        "Earnings Yield (E/P)", "Growth (g)",
        "E(Rm) Annual", "Rf Annual", "MRP Annual",
        "E(Rm) Monthly", "Rf Monthly", "MRP Monthly"
    ]].to_string(index=False))


=== Earnings Yield Approach (S&P 500 proxy) ===
label    pe_used  pe_value Earnings Yield (E/P) Growth (g) E(Rm) Annual Rf Annual MRP Annual E(Rm) Monthly Rf Monthly MRP Monthly
  LOW trailingPE 27.504107                3.64%      3.00%        6.64%     3.55%      3.09%         0.54%      0.29%       0.25%
 BASE trailingPE 27.504107                3.64%      4.50%        8.14%     3.55%      4.59%         0.65%      0.29%       0.36%
 HIGH trailingPE 27.504107                3.64%      6.00%        9.64%     3.55%      6.09%         0.77%      0.29%       0.48%
